In [ ]:
%config InlineBackend.figure_format = 'retina'
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dat = pd.read_csv('../input/database.csv')

In [ ]:
#When importing data, one record was imported in such a way as to shift over values by
#one variable starting at Last Name. This led to the creation of an additional variable.
#In the next cell, I will correct this issue:
dat[dat['Unnamed: 13'].notnull()]

In [ ]:
dat.loc[dat['First Name']=='Nadar ', 'First Name']='Nadar' #removes trailing whitespace in first name
dat.loc[dat['First Name']=='Nadar', 'Last Name'] = 'Hammett'
dat.loc[dat['First Name']=='Nadar', 'Age'] = 29
dat.loc[dat['First Name']=='Nadar', 'Rank'] = 'Firefighter/EMT'
dat.loc[dat['First Name']=='Nadar', 'Classification'] = 'Career'
dat.loc[dat['First Name']=='Nadar', 'Date of Incident'] = 'Dec 1, 2003'
dat.loc[dat['First Name']=='Nadar', 'Date of Death'] = 'Dec 1, 2003'
dat.loc[dat['First Name']=='Nadar', 'Cause Of Death'] = 'Impact'
dat.loc[dat['First Name']=='Nadar', 'Nature Of Death'] = 'No'
dat.loc[dat['First Name']=='Nadar', 'Duty'] = 'Trauma'
dat.loc[dat['First Name']=='Nadar', 'Activity']= 'Personal Vehicle Driver/Passenger'
dat.loc[dat['First Name']=='Nadar', 'Emergency']= 'Training'
dat.loc[dat['First Name']=='Nadar', 'Property Type']= 'Street/Road'

#drop the additional variable:

del dat['Unnamed: 13']

In [ ]:
dat['Date of Incident'] = pd.to_datetime(dat['Date of Incident'], errors='coerce')
dat['Date of Death'] = pd.to_datetime(dat['Date of Death'])

In [ ]:
#To get a sense of the firefighter casualties by classification, I binned some groups in the
#original data that appeared to be similar. 

vals = {'Wildland Full-Time': 'Wildland', 'Wildland Contract': 'Wildland', 
        'Wildland Part-Time': 'Wildland', 'Paid-on-Call': 'Part-Time', 'Part-Time (Paid)':
        'Part-Time'}

print("Breakdown of Firefighter casualties by Classification:")
dat.Classification.replace(vals.keys(), vals.values()).value_counts()

In [ ]:
#We take a subset of the data as specified and put it in object 'data':

data=dat.loc[(dat.Classification=='Career') | (dat.Classification=='Volunteer'), :].copy()

In [ ]:
data['Age'] = data['Age'].astype("float64")

from  matplotlib.ticker import FuncFormatter

f, ax = plt.subplots(figsize=(7,7))

career = data.loc[(data.Age.notnull()) & (data.Classification=='Career'), ['Date of Death', 
         'Age']].groupby(dat['Date of Death'].dt.year).Age.mean()

career.plot(kind='line', color='blue', ax=ax)

volunteer = data.loc[(data.Age.notnull()) & (data.Classification=='Volunteer'), ['Date of Death', 
         'Age']].groupby(dat['Date of Death'].dt.year).Age.mean()

volunteer.plot(kind='line', color='red', ax=ax)

ax.set_title("Average Age of Deceased Firefighters by Job Classification", fontsize=13, fontweight='bold')
ax.set_facecolor("white")
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
ax.set_ylim(bottom=0, top=60)
ax.set_xlabel("Year", fontweight='bold')
ax.set_ylabel("Age", fontweight='bold')
ax.fill_between(x=career.index, y1=career.values, interpolate=True)
ax.fill_between(x=volunteer.index, y1=career.values, y2=volunteer.values, facecolor='red',
                where=volunteer.values>career.values, interpolate=True, alpha=0.5)
ax.legend(labels=['career', 'volunteer']);

In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets


#The Causes of Death will be grouped according to these bins which seem to capture similar
#events.
bins = {'Impact': 'Impact/Vehicle Collision', 'Vehicle Collision': 'Impact/Vehicle Collision',
        'Trapped': 'Trapped/Exposure', 'Exposure': 'Trapped/Exposure', 
        'Collapse': 'Collapse/Fall', 'Fall': 'Collapse/Fall', 'Smoke': 'Trapped/Exposure',
        'Disorientation': 'Trapped/Exposure', 'Unknown': 'Other', 'Assault': 'Other',
        'Contact': 'Other'}

@interact(year=widgets.IntSlider(min=2000, max=2015, step=1, value=2000))
def annual(year):
     f, ax = plt.subplots(figsize=(12,6))
     plot = data.groupby([data['Date of Death'].dt.year, data['Classification'], 
                       data['Cause Of Death'].replace(bins.keys(), bins.values())
                         ])['Cause Of Death'].size().xs(year, 
                       level=0).unstack(level=1).plot(kind='bar', stacked=True, ax=ax)
     plot.set_title("Casualties by Cause of Death in "+ str(year), fontsize=20) 
     plot.set_facecolor("white")
     plot.set_xticklabels(plot.xaxis.get_majorticklabels(), rotation=2)
     plot.set_xlabel("")
     return plot
     
    